In [1]:
from __future__ import print_function, division
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import geopandas as gpd
import requests
import json
from io import StringIO
import io
import shapely

DEVELOPE = True

#### weather data - Local Climatological Data (LCD)

In [2]:
df = pd.read_csv('weather130101-171130.csv')
df.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (10,11,14,15,20,23,25,40,44,45,67,68,84,85,86,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,STATION,STATION_NAME,ELEVATION,LATITUDE,LONGITUDE,DATE,REPORTTPYE,HOURLYSKYCONDITIONS,HOURLYVISIBILITY,HOURLYPRSENTWEATHERTYPE,...,MonthlyMaxSeaLevelPressureTime,MonthlyMinSeaLevelPressureValue,MonthlyMinSeaLevelPressureDate,MonthlyMinSeaLevelPressureTime,MonthlyTotalHeatingDegreeDays,MonthlyTotalCoolingDegreeDays,MonthlyDeptFromNormalHeatingDD,MonthlyDeptFromNormalCoolingDD,MonthlyTotalSeasonToDateHeatingDD,MonthlyTotalSeasonToDateCoolingDD
0,WBAN:94728,NY CITY CENTRAL PARK NY US,42.7,40.77898,-73.96925,2013-01-01 00:51,FM-15,OVC:08 100,10.00,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
1,WBAN:94728,NY CITY CENTRAL PARK NY US,42.7,40.77898,-73.96925,2013-01-01 01:51,FM-15,OVC:08 60,10.00,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
2,WBAN:94728,NY CITY CENTRAL PARK NY US,42.7,40.77898,-73.96925,2013-01-01 02:51,FM-15,OVC:08 48,10.00,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
3,WBAN:94728,NY CITY CENTRAL PARK NY US,42.7,40.77898,-73.96925,2013-01-01 03:51,FM-15,OVC:08 46,10.00,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
4,WBAN:94728,NY CITY CENTRAL PARK NY US,42.7,40.77898,-73.96925,2013-01-01 04:51,FM-15,OVC:08 50,10.00,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
print ("Shape of data\n{}".format(df.shape))

Shape of data
(55996, 90)


In [4]:
df.drop(['STATION', 'STATION_NAME', 'ELEVATION', 'LATITUDE', 
              'LONGITUDE', 'REPORTTPYE', 'HOURLYSKYCONDITIONS', 
              'HOURLYVISIBILITY', 'HOURLYPRSENTWEATHERTYPE', 
              'HOURLYDRYBULBTEMPF', 'HOURLYDRYBULBTEMPC', 
              'HOURLYWETBULBTEMPF', 'HOURLYWETBULBTEMPC', 
              'HOURLYDewPointTempF', 'HOURLYDewPointTempC', 
              'HOURLYRelativeHumidity', 'HOURLYWindDirection',
              'HOURLYPrecip', 'MonthlyMaximumTemp',
       'MonthlyMinimumTemp', 'MonthlyMeanTemp', 'MonthlyAverageRH',
       'MonthlyDewpointTemp', 'MonthlyWetBulbTemp',
       'MonthlyAvgHeatingDegreeDays', 'MonthlyAvgCoolingDegreeDays',
       'MonthlyStationPressure', 'MonthlySeaLevelPressure',
       'MonthlyAverageWindSpeed', 'MonthlyTotalSnowfall',
       'MonthlyDeptFromNormalMaximumTemp', 'MonthlyDeptFromNormalMinimumTemp',
       'MonthlyDeptFromNormalAverageTemp', 'MonthlyDeptFromNormalPrecip',
       'MonthlyTotalLiquidPrecip', 'MonthlyGreatestPrecip',
       'MonthlyGreatestPrecipDate', 'MonthlyGreatestSnowfall',
       'MonthlyGreatestSnowfallDate', 'MonthlyGreatestSnowDepth',
       'MonthlyGreatestSnowDepthDate', 'MonthlyDaysWithGT90Temp',
       'MonthlyDaysWithLT32Temp', 'MonthlyDaysWithGT32Temp',
       'MonthlyDaysWithLT0Temp', 'MonthlyDaysWithGT001Precip',
       'MonthlyDaysWithGT010Precip', 'MonthlyDaysWithGT1Snow',
       'MonthlyMaxSeaLevelPressureValue', 'MonthlyMaxSeaLevelPressureDate',
       'MonthlyMaxSeaLevelPressureTime', 'MonthlyMinSeaLevelPressureValue',
       'MonthlyMinSeaLevelPressureDate', 'MonthlyMinSeaLevelPressureTime',
       'MonthlyTotalHeatingDegreeDays', 'MonthlyTotalCoolingDegreeDays',
       'MonthlyDeptFromNormalHeatingDD', 'MonthlyDeptFromNormalCoolingDD',
       'MonthlyTotalSeasonToDateHeatingDD',
       'MonthlyTotalSeasonToDateCoolingDD'], axis=1, inplace=True)

In [5]:
df.columns

Index(['DATE', 'HOURLYWindSpeed', 'HOURLYWindGustSpeed',
       'HOURLYStationPressure', 'HOURLYPressureTendency',
       'HOURLYPressureChange', 'HOURLYSeaLevelPressure',
       'HOURLYAltimeterSetting', 'DAILYMaximumDryBulbTemp',
       'DAILYMinimumDryBulbTemp', 'DAILYAverageDryBulbTemp',
       'DAILYDeptFromNormalAverageTemp', 'DAILYAverageRelativeHumidity',
       'DAILYAverageDewPointTemp', 'DAILYAverageWetBulbTemp',
       'DAILYHeatingDegreeDays', 'DAILYCoolingDegreeDays', 'DAILYSunrise',
       'DAILYSunset', 'DAILYWeather', 'DAILYPrecip', 'DAILYSnowfall',
       'DAILYSnowDepth', 'DAILYAverageStationPressure',
       'DAILYAverageSeaLevelPressure', 'DAILYAverageWindSpeed',
       'DAILYPeakWindSpeed', 'PeakWindDirection', 'DAILYSustainedWindSpeed',
       'DAILYSustainedWindDirection'],
      dtype='object')

In [6]:
df.count()

DATE                              55996
HOURLYWindSpeed                   50749
HOURLYWindGustSpeed                9362
HOURLYStationPressure             53604
HOURLYPressureTendency            14083
HOURLYPressureChange               2423
HOURLYSeaLevelPressure            42318
HOURLYAltimeterSetting            53660
DAILYMaximumDryBulbTemp            3362
DAILYMinimumDryBulbTemp            3361
DAILYAverageDryBulbTemp            3361
DAILYDeptFromNormalAverageTemp     3359
DAILYAverageRelativeHumidity       1548
DAILYAverageDewPointTemp           1584
DAILYAverageWetBulbTemp            1584
DAILYHeatingDegreeDays             3361
DAILYCoolingDegreeDays             3361
DAILYSunrise                      55996
DAILYSunset                       55996
DAILYWeather                        776
DAILYPrecip                        2388
DAILYSnowfall                      1794
DAILYSnowDepth                     1794
DAILYAverageStationPressure        1760
DAILYAverageSeaLevelPressure       1582
